# Using Curves with an Index and Inflation Instruments

This page exemplifies the ways of constructing *Curves* dealing with inflation and inflation linked products.
E.g. ``IndexFixedRateBond``, ``ZCIS`` and ``IIRS``.

> **Key Points**
> 
> - A `Series` of index values uses real data, with a zero month lag and the month is indexed to the 1st of the month.
> - A `Curve` can have any ``index_lag`` but best practice is to set it to zero to be consistent with ``index_fixings``.
> - A `Curve` can be calibrated by forecast RPI/CPI index values in a `Solver` using the `Value` *Instrument* type.

### Begin with a simple case **without** a ``Curve`` or any ``index_fixings`` 

This case uses an `IndexFixedRateBond` which has **two** coupon periods. The bond that is created below is fictional. It has the normal 3 month ``index_lag``, *'daily'* ``index_method`` for interpolation and the ``index_base`` for the *Instrument* is set to 381.0.

Its **cashflows** can be generated but are **not** fully formed becuase we are lacking information about the index: UK RPI.

In [ ]:
from rateslib import *
from pandas import Series, DataFrame

today = dt(2025, 5, 12)

ukti = IndexFixedRateBond(
    effective=dt(2024, 5, 27),
    termination=dt(2025, 5, 27),
    fixed_rate=2.0,
    notional=-10e6,
    index_base=381.0,
    index_method="daily",
    index_lag=3,
    spec="uk_gb"
)

In [ ]:
ukti.cashflows()

### Adding ``index_fixings`` as a ``Series``

Becuase this bond has a 3 month ``index_lag`` the most recent print required to determine all the cashflows is the RPI index for **March 2025**. In *rateslib* the RPI value for March must be indexed to 1st March, i.e. ``index_fixings`` as a *Series*  **must have a zero lag**. The below are **real** published RPI prints for the UK. (Note that Bloomberg will index these to the end of the month instead of the start of the month)

In [ ]:
from pandas import DataFrame
fixings.add("RPI_series", DataFrame([
    [dt(2024, 2, 1), 381.0],
    [dt(2024, 3, 1), 383.0],
    [dt(2024, 4, 1), 385.0],
    [dt(2024, 5, 1), 386.4],
    [dt(2024, 6, 1), 387.3],
    [dt(2024, 7, 1), 387.5],
    [dt(2024, 8, 1), 389.9],
    [dt(2024, 9, 1), 388.6],
    [dt(2024, 10, 1), 390.7],
    [dt(2024, 11, 1), 390.9],
    [dt(2024, 12, 1), 392.1],
    [dt(2025, 1, 1), 391.7],
    [dt(2025, 2, 1), 394.0],
    [dt(2025, 3, 1), 395.3]
], columns=["month", "rate"]).set_index("month")["rate"])
fixings["RPI_series"]

If the bond is recreated supplying the ``index_fixings`` the cashflows will be fully formed. Additionally we can use the same ``RPI_series`` to set the ``index_base`` value.

For good order the ``index_base`` is expected to be (and will be visible in one of the columns in cashflows):

$$ RPI_{Feb} + (RPI_{Mar} - RPI_{Feb}) * (27-1) / 31 = 382.677.. $$

In [ ]:
ukti = IndexFixedRateBond(
    effective=dt(2024, 5, 27),
    termination=dt(2025, 5, 27),
    fixed_rate=2.0,
    notional=-10e6,
    index_method="daily",
    index_lag=3,
    index_fixings="RPI_series",
    spec="uk_gb"
)

In [ ]:
ukti.cashflows()

### Adding a discount *Curve*

The **npv** of the cashflows, and of the bond are still not available becuase there is no discount curve. Let's add one. Note that its initial date is, as usual, set to **today**.

In [ ]:
disc_curve = Curve({today: 1.0, dt(2029, 1, 1): 0.95})

There is now sufficient information to price any aspect of this bond becuase the ``index_fixings`` are determined and the discount *Curve* can value the future cashflows.

The prices shown below will be for the standard T+1 settlement under the ``uk_gb`` default ``spec``.

In [ ]:
ukti.cashflows(curves=[None, disc_curve])

In [ ]:
ukti.rate(curves=[None, disc_curve], metric="clean_price")

In [ ]:
ukti.rate(curves=[None, disc_curve], metric="index_clean_price")

### Adding a forecast *Index Curve*

Now we will add a forecast *Index Curve*. *Rateslib* allows *Curves* to be parametrised according to their own ``index_lag``, but the most natural definition is to define a *Curve* with a **zero index lag**, consistent with the *Series*. This is more transparent.

Our *Curve* will start as of the last available RPI value date, indexed to that level. I.e. starting at 1st March with a base value of 395.3.

We calibrate the Curve, for this example, not with market instruments but instead directly with *Index* ``Values`` we wish to use.

In [ ]:
index_curve = Curve(
    nodes={
        dt(2025, 3, 1): 1.0,
        dt(2025, 4, 1): 1.0, 
        dt(2025, 5, 1): 1.0,
        dt(2025, 6, 1): 1.0,
        dt(2025, 7, 1): 1.0,
    },
    index_lag=0,
    index_base=395.3,
    id="ic",
)
solver = Solver(
    curves=[index_curve],
    instruments=[
        Value(effective=dt(2025, 4, 1), metric="index_value", curves="ic"),
        Value(effective=dt(2025, 5, 1), metric="index_value", curves="ic"),
        Value(effective=dt(2025, 6, 1), metric="index_value", curves="ic"),
        Value(effective=dt(2025, 7, 1), metric="index_value", curves="ic"),
    ],
    s=[396, 397.1, 398, 398.8],
    instrument_labels=["Apr", "May", "Jun", "Jul"],
)

### An Instrument with mixed ``index_fixings`` and forecast fixings

Now we can create an *Instrument* which requires both historical fixings and forecast values. Changing the dates of the fictional bond to end in, say, September 2025, requires the fixings forecast on the curve for June and July. Note we choose to add the ``curves`` directly at *Instrument* initialisation.

In [ ]:
ukti = IndexFixedRateBond(
    effective=dt(2024, 9, 16),
    termination=dt(2025, 9, 16),
    fixed_rate=3.0,
    notional=-15e6,
    index_method="daily",
    index_lag=3,
    index_fixings="RPI_series",
    spec="uk_gb",
    curves=[index_curve, disc_curve]
)

In [ ]:
ukti.cashflows()

### Bonus: Risk to RPI prints.

Actually the way we have constructed this *Index Curve* using the *Solver* means we can directly extract monetary sensitivities to the RPI index values

In [ ]:
ukti.delta(solver=solver)

For the 15mm GBP bond owned here, for each unit of the RPI print that comes above the supposed values of 398.0 and 398.8 the PnL will increase by £19.5k.
Thus a +0.1% MoM surpise in June shifts up the values in June and July both by about 0.4. This would be expected to affect the NPV by £15.6k.

In [ ]:
pv_0 = ukti.npv()
pv_0

In [ ]:
solver.s = s=[396, 397.1, 398.4, 399.2]  # <-- Shift the Jun and Jul prints both up by 0.4, i.e. 0.1% MOM suprise in Jun.
solver.iterate()

In [ ]:
pv_1 = ukti.npv()
pv_1 - pv_0

### Other Instruments and Other Lags

We can use the objects already created to price other *Instruments*. We directly construct an ``IndexFixedLeg`` below as an example with an ``index_lag`` of 2.

In [ ]:
ifl = FixedLeg(
    schedule=Schedule(dt(2024, 12, 1), "8m", "M"),
    fixed_rate=1.0,
    notional=-15e6,
    convention="30360",
    index_fixings="RPI_series",
    index_lag=2,
    index_method="monthly",
    currency="gbp"
)

The cashflows below show the *index values* beginning with the November 2024 RPI value progressing through to the known March 2025 value and then adopting the values forecast by the *Curve*.

In [ ]:
ifl.cashflows(index_curve=index_curve, disc_curve=disc_curve)

In [ ]:
solver.delta(ifl.npv(index_curve=index_curve, disc_curve=disc_curve, local=True))